In [2]:
import numpy as np
import pandas as pd
from tqdm import trange

In [2]:


class UserBased:
    mu: np.ndarray
    sim: np.ndarray

    def __init__(self, zero_mean: bool = True, beta: int = 1, idf: bool = False, verbosity: int = 0):
        """

        :param zero_mean:
        :param beta: Discounting parameter
        :param idf: Enable inverse document frequency management
        """
        self.zero_mean = zero_mean
        self.beta = beta
        self.idf = idf
        self.verbosity = verbosity

    def fit(self, r: np.ndarray):
        m, n = r.shape
        if self.zero_mean:
            self.mu = np.nanmean(r, axis=1)
        else:
            self.mu = np.zeros(m)

        self.sim = np.zeros((m, m))

        if self.idf:
            idf = np.log(1 + m / (~np.isnan(r)).sum(axis=0))
        else:
            idf = np.ones(n)

        if self.verbosity > 0:
            print(idf)

        for i in trange(m):
            for j in range(m):
                mask = ~np.isnan(r[i, :]) & ~np.isnan(r[j, :])

                si = r[i, mask] - self.mu[i]
                sj = r[j, mask] - self.mu[j]

                self.sim[i][j] = (si * sj * idf[mask]).sum() / (
                        np.sqrt((idf[mask] * (si ** 2)).sum()) * np.sqrt((idf[mask] * (sj ** 2)).sum()))

                total_intersection = mask.sum()

                self.sim[i][j] *= min(total_intersection, self.beta) / self.beta

        return self.sim

    def predict(self, r: np.array, u: int, top_k: int = 3) -> np.ndarray:
        """

        :param r: Rating matrix
        :param u: User u
        :param top_k: Top k neighbourhood
        :return: Calculated Rating of each item
        """

        _, n = r.shape

        score = np.zeros(n)

        for j in trange(n):
            score[j] = self.predict1(r, u, j, top_k)

        return score

    def predict1(self, r: np.array, u: int, j: int, top_k: int = 3) -> float:
        _, n = r.shape

        users_rated_j = np.nonzero(~np.isnan(r[:, j]))[0]

        topk_users = users_rated_j[self.sim[u, users_rated_j].argsort()[::-1][:top_k]]

        mean_centered_topk_user_rate = r[topk_users, j] - self.mu[topk_users]

        w = self.sim[u, topk_users]

        return np.dot(mean_centered_topk_user_rate, w) / np.abs(w).sum() + self.mu[u]



In [10]:
def read_data(select):
    if select == "small":
        return np.array([[7, 6, 7, 4, 5, 4],
              [6, 7, np.nan, 4, 3, 4],
              [np.nan, 3, 3, 1, 1, np.nan],
              [1, 2, 3, 3, 3, 4],
              [1, np.nan, 1, 2, 3, 3]])
    elif select == "large":
        df = pd.read_csv('https://files.grouplens.org/datasets/movielens/ml-100k/u.data', delimiter=r'\t', engine='python',names=['user_id', 'item_id', 'rating', 'timestamp'])
        return df.pivot(index='user_id', columns='item_id', values='rating').values
    else:
        print("selection not recognized!")


#%

In [8]:
def create_test_data(r):
    irow, jcol = np.where(~np.isnan(r))
    idx = np.random.choice(np.arange(3), 3, replace=False)
    test_irow = irow[idx]
    test_jcol = jcol[idx]

    r_copy = r.copy()
    for i in test_irow:
        for j in test_jcol:
            r_copy[i][j] = np.nan
    return r_copy, test_irow, test_jcol

In [21]:
user = UserBased()
sim_u = user.fit(r_copy)
sim_i = user.fit(r_copy.T)


  0%|                                                                                          | 0/943 [00:00<?, ?it/s]C:\Users\berda\AppData\Local\Temp\ipykernel_11816\673340450.py:44: RuntimeWarning: invalid value encountered in double_scalars
  self.sim[i][j] = (si * sj * idf[mask]).sum() / (
100%|██████████████████████████████████████████████████████████████████████████████| 1682/1682 [05:52<00:00,  4.77it/s]


In [41]:
def get_users_rated_item(r_not_nan, item):
    users = []
    for i in r_not_nan:
        if i[1] == item:
            users.append(i[0])
    return users

def get_items_rated_by_user(r_not_nan, user):
    items = []
    for i in r_not_nan:
        if i[0] == user:
            items.append(i[1])
    return items

def get_topk_users(users_rated_item, u, k):
    topk = {}
    for i in users_rated_item:
        topk[i] = sim_u[i][u]
    topk = dict(sorted(topk.items(), key=lambda item: item[1]))
    topk.popitem()

    return list(reversed(list(topk)))[:k]


def get_topk_items(items_rated_by_user, u, k):
    topk = {}
    for i in items_rated_by_user:
        topk[i] = sim_i[i][u]
    topk = dict(sorted(topk.items(), key=lambda item: item[1]))
    topk.popitem()

    return list(reversed(list(topk)))[:k]

In [97]:





def model1(r, k, alpha, max_iter):
    r_not_nan = np.argwhere(~np.isnan(r))
    m,n = r.shape
    r_pred = np.empty(r.shape)
    r_pred[:] = np.nan
    mu = np.nanmean(r, axis=1)

    wu = np.random.rand(m, k)
    wj = np.random.rand(n, k)
    for iteration in range(max_iter):
        for u,j in r_not_nan:
            top_k_users = get_topk_users(users_rated_item=get_users_rated_item(r_not_nan, j), u=u, k=k)
            top_k_items = get_topk_items(items_rated_by_user=get_items_rated_by_user(r_not_nan, u), u=j, k=k)
            r_pred[u,j] = mu[u] + np.dot(wu[u, :], (r[top_k_users, j]) - mu[top_k_users]) + np.dot(wj[j, :], r[u, top_k_items])
            g_wu = -1 * np.dot((r[u, j] - r_pred[u,j]), (r[top_k_users, j]) - mu[top_k_users])
            g_wj = -1 * np.dot((r[u, j] - r_pred[u, j]), (r[u, top_k_items]))
            wu[u, :] = wu[u, :] - alpha * g_wu
            wj[j, :] = wj[j, :] - alpha * g_wj
    return wu, wj



In [13]:
def loss(r, r_pred, r_not_nan):
    loss = []
    for i,j in r_not_nan:
        loss.append(np.power((r[i, j] - r_pred[i, j]), 2))
        s = sum(loss)
    return s/2

def model2(r, alpha, max_iter):
    r_not_nan = np.argwhere(~np.isnan(r))
    m,n = r.shape
    r_pred = np.empty(r.shape)
    r_pred[:] = np.nan
    bu = np.random.rand(m)
    bi = np.random.rand(n)
    for iteration in trange(max_iter):
        for u, j in r_not_nan:
            r_pred[u, j] = bu[u] + bi[j]
        if iteration%(max_iter/10) == 0:
            print(loss(r, r_pred, r_not_nan))

        g_bu = np.zeros(m)
        g_bi = np.zeros(n)
        for u, j in r_not_nan:
            g = bu[u] + bi[j] - r[u, j]
            g_bu[u] += g
            g_bi[j] += g
        bu_prev = np.copy(bu)
        bi_prev = np.copy(bi)
        bu = bu - g_bu * alpha
        bi = bi - g_bi * alpha

        if np.linalg.norm(bu - bu_prev) < 0.0001 and np.linalg.norm(bi - bi_prev):
            print(iteration, "iterations")
            return bu, bi


    return bu, bi

def test2(bu, bi, r_true,test_irow, test_jcol):
    err = []
    for i in test_irow:
        for j in test_jcol:
            err.append(((bu[i] + bi[j]) - r_true[i , j])**2)
    print(f"RMSE: {np.sqrt(np.nanmean(np.array(err)))}")

r = read_data("large")
r_copy, test_irow, test_jcol = create_test_data(r)
bu, bi = model2(r_copy, alpha=0.001, max_iter=1000)
test2(bu, bi, r, test_irow, test_jcol)

  0%|                                                                                         | 0/1000 [03:46<?, ?it/s]


KeyboardInterrupt: 

In [26]:
wu, wj = model1(r_copy,k=2, alpha=0.001, max_iter=1)
mu = np.nanmean(r, axis=1)
r_pred = np.empty(r.shape)
r_pred[:] = np.nan
err = []
k=2
for u, j in zip(test_irow, test_jcol):
    top_k_users = get_topk_users(users_rated_item=get_users_rated_item(r_not_nan, j), u=u, k=k)
    top_k_items = get_topk_items(items_rated_by_user=get_items_rated_by_user(r_not_nan, u), u=j, k=k)
    r_pred[u,j] = mu[u] + np.dot(wu[u, :], (r[top_k_users, j]) - mu[top_k_users]) + np.dot(wj[j, :], r[u, top_k_items])
    y = r[u, j]

    err.append((r_pred - y) ** 2)

print(f"RMSE: {np.sqrt(np.nanmean(np.array(err)))}")









RMSE: nan


C:\Users\berda\AppData\Local\Temp\ipykernel_11816\2230093236.py:15: RuntimeWarning: Mean of empty slice
  print(f"RMSE: {np.sqrt(np.nanmean(np.array(err)))}")
